# Example 7: Measuring worm length

Body length is a commonly measured trait. However, live animals are typically not well behaved in front of camera: typically the body is not aligned straight, which makes length measure difficult and sometimes time consuming. Some computer vision can help here, such as skeletonization (https://en.wikipedia.org/wiki/Topological_skeleton) for an acurate semi automatic approach, and polyline drawing for a less acurate, manual aproach.

Here I demonstrate how to automatically measure the length of california blackworms (*Lumbriculus variegatus*) in 6 well plates by dragging a mask around them. A future implementation will like include a variant of [Hough transformation](https://en.wikipedia.org/wiki/Hough_transform) to automatically detect the circles of the wellplates for a full automatic workflow. 

**Note:** For details on the differen workflows (prototyping-, low throughput- and high throughout) see [Tutorial 2](tutorial_2_phenopype_workflow.ipynb). Not all examples include the prototyping workflow.

<div class="row; text-align: left">
    
<div class="col-md-6">
    
![Before](_assets/ex7_before.jpg)
    
**Input** - Worms placed in 6 well plates
</div>
<div class="col-md-6">

![After](_assets/ex7_after.jpg)
    
**Results** - extracted body contour (green area) and body length (red line) as a topological skeleton
</div>
</div>


## Low throughput

First we load the image as a container. If you have saved data before with a specific `save_suffix` (recommended for reproducibility and to not overwrite files) you can reload it if you have provided it at the point of container creation in `load_image`:

In [1]:
import phenopype as pp

In [2]:
filepath = r"./images/worms.jpg"
image, img_data = pp.load_image(filepath, df=True)
pp.show_image(image)

First we drag a rectangle around each worm in each well. Future releases will include "circle"-shaped mask tools, and the opportunity to automatically find circles using hough transform - OpenCV is already set up to do so: https://docs.opencv.org/3.4/d4/d70/tutorial_hough_circle.html

In [4]:
df_masks = pp.preprocessing.create_mask(image) 

- creating mask


Next step is to perform the segmentation. To adjust vor variable brightness inside the wells we use the `"adaptive"` `threshold` method, and afterwards, `find_contours`. If the worms touch the border of the well, or their own body, you can directly manipulate the binary image using the draw function (two clicks draw a line, or use the argument `tool="rectangle"`. Sometimes a worm may overlap with itself - in that case, use `polylines` to measure it manually.

In [5]:
image_bin = pp.segmentation.threshold(image, 
                          df_masks=df_masks,
                          method="adaptive", 
                          blocksize=49, 
                          constant=5)

- including pixels from 6 drawn masks 


In [6]:
## pp.segmentation(draw) # uncomment this if you need to manually separate worms from the well-border
df_contours = pp.segmentation.find_contours(image_bin, 
                                            min_diameter=50)
## pp.visualization.polylines(ct) # uncomment this if you need to manually measure a worm
# this will measure the longest distance in a curvy object
df_contours = pp.measurement.skeletonize(image_bin, 
                                         img_data,
                                         df_contours) 

- found 6 contours that match criteria


Next we draw the output, and save the results (including a visualization).

In [7]:
canvas = pp.visualization.select_canvas(image, 
                                        canvas="raw") # first select a background (try "binary")
canvas = pp.visualization.draw_contours(canvas, 
                                        df_contours) # this draws both the contour (green) and the skeleton
canvas = pp.visualization.draw_masks(canvas, 
                                     df_masks) # will be skipped if loaded before

- raw image
drawing mask: mask1


In [8]:
pp.show_image(canvas)

In [9]:
pp.export.save_masks(df_masks, 
                     dirpath=r"_temp/output/ex7") # 
pp.export.save_contours(df_contours, 
                        dirpath=r"_temp/output/ex7")
pp.export.save_canvas(canvas, 
                      dirpath=r"_temp/output/ex7", resize=1) # resize=1 to have the original sized picture

- masks saved under _temp/output/ex7\masks.csv (overwritten).
- contours saved under _temp/output/ex7\contours.csv (overwritten).
- canvas saved under _temp/output/ex7\canvas.jpg (overwritten).


Should a worm superimpose itself it is not possible for the skeletonization algorithm to follow the shape through from beginning to end (e.g. in worm 3 in this example). In that case one has to manually draw a line along the worm using the `polylines` tool. 

<center>
<div style="width:300px; text-align: left" >
    
![Canvas masking](_assets/ex7_poly.png)
    
</div>
</center>

In [12]:
df_polylines = pp.measurement.polylines(canvas)

canvas = pp.visualization.draw_polylines(canvas, 
                                         df_polylines)

- draw polylines


In [14]:
pp.show_image(canvas)

In [15]:
pp.export.save_polylines(df_polylines, dirpath=r"_temp/output/ex7")

- polylines saved under _temp/output/ex7\polylines.csv (overwritten).


Alternatively to the `polylines` tool, one can also use the drawing tool if you just need to separte an endpoint or if the worm touches the border. This should be done on the binary image, *before* the contour are detected. 

## High throughput

As for the other examples I have created a template (`ex7`) with appropriate settings for the example. The template can be passed to the `pype` using `template="ex7"` - see below. 

In [16]:
import phenopype as pp

image_path = r"./images/worms.jpg"

pp.show_config_template("ex7")

SHOWING BUILTIN PHENOPYPE TEMPLATE ex7.yaml


- preprocessing:
  - create_mask
- segmentation:
  - threshold:
      method: adaptive
      blocksize: 49
      constant: 5
  - draw
  - find_contours:
      retrieval: ccomp
      min_diameter: 50
      min_area: 0
- measurement:
  - skeletonize
  - polylines
- visualization:
  - select_canvas:
      canvas: image
  - draw_contours:
      line_width: 2
      label_width: 1
      label_size: 1
      fill: 0.3
  - draw_masks
  - draw_polylines
- export:
  - save_contours:
      overwrite: true


In [17]:
pp.pype(image_path, 
        name="worms2",
        dirpath=r"_temp/output/ex7", 
        template="ex7")

Directory to save phenopype-container output set to parent folder of image:
D:\workspace\git\phenopype\tutorials\images
pype_config_worms2.yaml already exists - overwrite?
y: yes, file will be overwritten and loaded
n: no, existing file will be loaded instead
To load an existing file, use "config" instead of "template".y
New pype configuration created (ex7.yaml) from phenopype template:
d:\workspace\git\phenopype\phenopype\templates\ex7.yaml


------------+++ new pype iteration 2021:05:10 16:18:55 +++--------------


=== AUTOLOAD ===
- polylines_worms2.csv
- masks_worms2.csv
PREPROCESSING
create_mask
- mask with label mask1 already created (edit/overwrite=False)
SEGMENTATION
threshold
- including pixels from 2 drawn masks 
draw
- drawing
zero coordinates - redo drawing!
segmentation.draw: ValueError - No objects to concatenate
find_contours
- found 8 contours that match criteria
MEASUREMENT
skeletonize
polylines
- polylines already drawn (overwrite=False)
VISUALIZATION
select_canvas
- 

SystemExit: 

TERMINATE (by user)